In [1]:
import csv
import pandas as pd
import numpy as np
import sqlite3
import matplotlib as plot

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
# write your code here
with sqlite3.connect("../db.sqlite3") as con:
    cursor = con.cursor()
    data = cursor.execute(
        """SELECT restaurant_orderitem.id, restaurant_orderitem.quantity, restaurant_order.datetime, restaurant_product.name, restaurant_product.price
        FROM restaurant_orderitem
        JOIN restaurant_order ON restaurant_order.id = restaurant_orderitem.order_id
        JOIN restaurant_product ON restaurant_product.id = restaurant_orderitem.product_id"""
    )

with open("product_data.csv", "w") as data_csv:
    writer = csv.writer(data_csv)
    writer.writerow(["id", "quantity", "datetime", "name", "price"])
    writer.writerows(data)

df = pd.read_csv("product_data.csv")
df.info()

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
df.groupby("name")["quantity"].sum().sort_values(ascending=False).head(10).plot.pie(title="Top 10 popular products sold quantity", figsize=(7, 7), autopct="%.1f%%")

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df["TotalPrice"] = df["price"] * df["quantity"]
df.groupby("name")["TotalPrice"].sum().sort_values(ascending=False).head(10).plot.pie(title="Top 10 products which make the most money for restaurant", figsize=(7, 7), autopct="%.1f%%")

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df["Order Hour"] = pd.DatetimeIndex(df["datetime"]).hour
df.groupby("Order Hour")["TotalPrice"].sum().plot.bar(title="Best hours for order", figsize=(10, 6))

# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
days_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df["Day of the week"] = pd.Series(pd.date_range(start="2015-10-02", end="2019-08-03", freq="D")).dt.day_name()
df.groupby("Day of the week")["TotalPrice"].sum().reindex(days_week).plot.bar(title="The best day for orders", figsize=(14, 6))